In [15]:
import ollama
import json
import time
import requests
from typing import List, Dict, Set
import pandas as pd

AGENT_3 = 'mistral'
BEARER_TOKEN = "insert your semantic scholar key here"
SS_HEADERS = {"Authorization": f"Bearer {BEARER_TOKEN}"}

In [16]:
with open("output/agent2_ideas.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [17]:
# keep only rows where title is nonempty
df = df[df['title'].str.strip().astype(bool)]

# (optional) reset the index to go 0…N-1 again
df = df.reset_index(drop=True)

In [18]:
df.head(20)

,title,description,data_needs,idea_id
0,Arctic Anomaly Detection with Computer Vision,A deep learning system that detects and classi...,"[Satellite imagery, Arctic climate data, histo...",8_arctic_anomaly_detection_with_computer_vision
1,Climate Resilience Prediction using GANs,Generative adversarial networks (GANs) simulat...,"[Historical climate data, urban/agricultural d...",9_climate_resilience_prediction_using_gans
2,Sea Level Rise Prediction with Deep Learning,A deep learning model that forecasts regional ...,"[Ocean current measurements, satellite altimet...",10_sea_level_rise_prediction_with_deep_learning
3,Renewable Energy Output Forecasting,Machine learning algorithms predict solar and ...,"[Weather forecasts, satellite imagery, energy ...",11_renewable_energy_output_forecasting
4,Climate Policy Optimization with Reinforcement...,Reinforcement learning models simulate and rec...,"[Climate policy datasets, economic indicators,...",12_climate_policy_optimization_with_reinforcem...
5,Sustainable Deep Learning Models for Energy Ef...,Designs energy-efficient deep learning archite...,"[Model performance metrics, energy consumption...",13_sustainable_deep_learning_models_for_energy...
6,Climate Data Fusion for Enhanced Predictions,A multimodal deep learning framework that fuse...,"[Satellite data, ground sensor networks, ocean...",14_climate_data_fusion_for_enhanced_predictions
7,AI-Driven Carbon Footprint Analysis,A deep learning system that tracks and predict...,"[Emissions inventories, supply chain data, ene...",15_ai-driven_carbon_footprint_analysis
8,Geoengineering Impact Assessment with Simulations,Machine learning models simulate the environme...,"[Atmospheric chemistry data, climate simulatio...",16_geoengineering_impact_assessment_with_simul...
9,Climate Tipping Points Prediction with Predict...,Predictive analytics models identify early war...,"[Historical climate records, tipping point ind...",17_climate_tipping_points_prediction_with_pred...


In [19]:
ideas = df["title"].tolist()
dict_of_title_and_idea_ids = df.set_index("title")["idea_id"].to_dict()

In [20]:
def query_ollama_model(model_name, prompt_for_keywords):
    try:
        print(f"Sending prompt to Ollama model: {model_name}...")
        response = ollama.chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': prompt_for_keywords,
                },
            ]
        )
        print("Received response from Ollama.")
        return response['message']['content']
    except Exception as e:
        return f"An error occurred while communicating with Ollama: {e}"

def get_ollama_response(prompt_for_keywords,model_name):

    # --- Execution ---
    print(f"Requesting keywords from Ollama for model: {model_name}...")
    
    keywords_response = query_ollama_model(model_name, prompt_for_keywords)
    return keywords_response

In [21]:
key_ideas_generation_prompt_1 = f"""
You are an AI Research Synthesizer. Your primary objective is to analyze a collection of `n` research paper abstracts, all related to a common theme or a specific research idea within <Topic>Climate Change and Deep Learning</Topic>. Your task is to distill these abstracts into a single, unified set of key bullet points that capture the holistic picture and core gist of the combined information. These points should highlight the most significant, recurring, or foundational insights that emerge when considering all abstracts together.

**Instructions for Synthesizing Collective Key Points:**

1.  **Comprehensive Review:** Thoroughly read and analyze ALL `n` provided abstracts to understand the full scope of information.
2.  **Identify Overarching Themes & Connections:** Look for:
    *   Common research questions, objectives, or problems addressed across multiple abstracts.
    *   Recurring methodologies, techniques, or datasets employed.
    *   Converging findings or consistent conclusions that appear in several abstracts.
    *   Complementary information where different abstracts contribute unique pieces to a larger puzzle.
    *   The overall narrative or argument these abstracts collectively support regarding the central theme or idea.
3.  **Synthesize, Don't Just Aggregate:** Your goal is not to pick one point from each abstract. Instead, formulate new summary points that represent the *synergistic understanding* gained from all abstracts. A single bullet point might draw from concepts mentioned in multiple abstracts.
4.  **Focus on the Core Gist:** The bullet points should represent the most critical and impactful insights that define the collective evidence or understanding presented. What are the absolute must-know takeaways if someone were to understand the essence of these `n` abstracts as a whole?
5.  **Conciseness and Clarity:** Each bullet point should be a clear, concise phrase or a short, impactful sentence.
6.  **Number of Points:** Aim for a focused list of 5-7 key bullet points in total for the entire set of abstracts. The exact number can vary based on the richness and diversity of the input, but the goal is a high-level synthesis.
7.  **Holistic Perspective:** The final list of bullet points should read as a coherent summary of the combined knowledge, not a disjointed collection.
8.  **Output Formatting (CRUCIAL):**
    *   Provide a single, unified list of bullet points.
    *   Use standard bullet characters (e.g., `*`, `-`, or `•`).
    *   Do NOT provide separate summaries for each abstract.
    *   Do NOT include any introductory phrases (e.g., "Here are the synthesized key points...") or concluding remarks, other than the single bulleted list.

**Input Abstracts:**

You will now be provided with `n` abstracts. Please process them *collectively* according to the instructions above to generate a *single list* of synthesized key points.
--
"""

key_ideas_generation_prompt_2 = """
**Your Task:**
Generate a single, unified list of 5-7 key bullet points that synthesize the core gist and holistic picture from ALL `n` abstracts provided above. Adhere strictly to all instructions, especially regarding the synthesis approach and output formatting.
"""

In [22]:
def search_papers(query: str, limit: int = 3) -> List[Dict]:
    """Search for papers using the given query."""
    SEARCH_URL = "https://api.semanticscholar.org/graph/v1/paper/search"

    search_params = {
        "query": query,
        "limit": limit,
        "fields": "paperId,title,externalIds,abstract"
    }

    try:
        resp = requests.get(SEARCH_URL, params=search_params, headers=SS_HEADERS)
        resp.raise_for_status()
        return resp.json().get("data", [])
    except requests.exceptions.RequestException as e:
        print(f"Search error for query '{query}': {e}")
        return []


# ——— 4) Function to get paper abstract from different sources ———
def get_paper_abstract(paper: Dict) -> str:
    """Attempt to get abstract from Semantic Scholar, then OpenAlex if needed."""
    # Check if we already have the abstract
    abstract = paper.get("abstract")
    if abstract:
        return abstract

    # If not, try to fetch it from Semantic Scholar
    DETAIL_URL = "https://api.semanticscholar.org/graph/v1/paper/"
    pid = paper["paperId"]

    try:
        detail_params = {"fields": "abstract,externalIds"}
        r = requests.get(DETAIL_URL + pid, params=detail_params, headers=SS_HEADERS)
        r.raise_for_status()
        data = r.json()
        abstract = data.get("abstract")

        # If SS returned no abstract, try OpenAlex
        if not abstract:
            ext = data.get("externalIds") or paper.get("externalIds", {})
            oa_id = ext.get("OpenAlex") if ext else None

            if oa_id:
                oa_url = f"https://api.openalex.org/works/{oa_id}"
                oa_r = requests.get(oa_url)
                if oa_r.ok:
                    oa_data = oa_r.json()
                    inv_idx = oa_data.get("abstract_inverted_index") or {}
                    if inv_idx:
                        # Reconstruct plain text abstract from inverted index
                        pos_map = {}
                        for word, positions in inv_idx.items():
                            for pos in positions:
                                pos_map[pos] = word
                        # Build ordered list of words
                        abstract = " ".join(
                            pos_map[i] for i in range(len(pos_map))
                            if i in pos_map
                        )

        return abstract or "(no abstract available)"

    except requests.exceptions.RequestException as e:
        print(f"Error fetching abstract for paper {pid}: {e}")
        return "(error retrieving abstract)"


# ——— 5) Main function to run the entire process ———
def main():

    # Track unique papers by ID to avoid duplicates
    unique_papers: Dict[str, Dict] = {}
    papers_per_query: Dict[str, Set[str]] = {}
    idea_bullet_Points = {}
    # Process each permutation
    for i, query in enumerate(ideas):
        print(f"Processing IDEA {i + 1}/{len(ideas)}: {query}")

        # Search for papers
        papers = search_papers(query)
        papers_per_query[query] = set()
        all_abstracts = ""
        for idx,paper in enumerate(papers):
            pid = paper["paperId"]
            papers_per_query[query].add(pid)

            # Skip if we already have this paper
            if pid in unique_papers:
                continue

            # Get the abstract if not already included
            if not paper.get("abstract"):
                paper["abstract"] = get_paper_abstract(paper)
            print(f"Found paper: {paper['title']} (ID: {pid})")
            all_abstracts += f"**Abstract {idx+1}:**\n {paper['abstract']}\n"
            print("=" * 80)
            unique_papers[pid] = paper
            time.sleep(1)
        key_ids_prompt_to_ollma = key_ideas_generation_prompt_1 + all_abstracts + key_ideas_generation_prompt_2
        bullet_points = get_ollama_response(key_ids_prompt_to_ollma, AGENT_3)

        idea_bullet_Points[dict_of_title_and_idea_ids[query]] = bullet_points
        time.sleep(1)
    with open(f"output/task3_idea_bullet_points.json", "w") as f:
        json.dump(idea_bullet_Points, f, indent=4)
    # Save the idea bullet points to a CSV file
    df = pd.DataFrame(idea_bullet_Points.items(), columns=["idea_id", "bullet_points"])
    df.to_csv(f"output/task3_idea_bullet_point.csv", index=False)
    

if __name__ == "__main__":
    main()


Processing IDEA 1/20: Arctic Anomaly Detection with Computer Vision
Found paper: Pedestrian Equipment Anomaly Detection with Computer Vision in Warehouses (ID: 734e724119e8a7920ef161c1b814bb492d87e261)
Found paper: Anomaly Detection in Industrial Quality Control with Computer Vision and Deep Learning (ID: 6554eb2d0b65a575d416f5c3de98c5201ec877ed)
Found paper: Research on the Application of Deep Learning-Based Computer Vision in Anomaly Detection in Communication Networks (ID: 7b13d8795a36c6f09e172836331435af4630ab35)
Requesting keywords from Ollama for model: mistral...
Sending prompt to Ollama model: mistral...
Received response from Ollama.
Processing IDEA 2/20: Climate Resilience Prediction using GANs
Found paper: AI-Driven Breeding Enhances Stress Tolerance in High-Elevation Extremophytes: A Proof-of-Concept Study with Cross-Component Validation (ID: cf4fa552dfd1a2f4e8b5f0d5426a9e381057cd41)
Found paper: Crop yield prediction through machine learning: A path towards sustainable agr